# Assignment #4 - Reinforcement Learning

### <font color="red"> DUE: Nov 26 (Tuesday) 11:00 pm </font>  

<font color="blue"> Anusha Vudathu </font>

# I. Overview

Describe the objective of this assignment. You can briefly state how you accompilsh it.
The objective of this assignment is to create an agent to play the blackjack game. 

Rummy is a card game where the goal is to obtain cards that are having the same number. In this game the players will play with each other and try to win over others. 

We can use any of the below given two methods for the agent.

- SARSA
- Q-learning

SARSA - State–action–reward–state–action (SARSA) is an algorithm for learning a Markov decision process policy, used in the reinforcement learning area of machine learning. It was proposed by Rummery and Niranjan in a technical note with the name "Modified Connectionist Q-Learning" (MCQ-L).

For any finite Markov decision process (FMDP), Q-learning finds a policy that is optimal in the sense that it maximizes the expected value of the total reward over any and all successive steps, starting from the current state.

I have used Q-learning method to train the model. 



# II. Problems 



## Rummy (Our Way)
![](https://upload.wikimedia.org/wikipedia/commons/9/9a/3_playing_cards.jpg)



### Description:
Rummy is mostly played by 4 players where each player get 10 cards and rest of placed as closed deck.  

But for this assignment to keep it simple, we have changed some rules, reduced the deck of cards and distribution of the cards among each player. Otherwise, the our Q table would be large to handle on our local machines.  

So in this game each player will get 3 cards randomly from 'A' to '5'. With each turn you have to pick a card either from the pile of closed or open cards depending on the cards you have. After that you have to select a card to drop to the pile from your deck. Your goal is to make all of your 3 cards of same number. As soon as one player reaches the goal state, it will meld the cards and game will stop and all other player will get score as the sum of their cards in hand. Player with minimum score will win the game.  

Each player will play for the 10 rounds and if nobody reach to the meld stage then each will player will get score equivalent to the sum of their cards in hand.

Player with the lowest score wins the game.



### <font color="red"> Game Introduction and Rules: </font>

You can only till max turns which is now 20. If you are able to meld your cards, you win otherwise you lose points as per your stash score.

Lets Simulate a Game Round :
The Goal of this game is to collect cards with the same rank.
Naming Conventions to follow H : Hearts, S : Spades, D : Diamonds, C : Clubs. Stash/Hand are the cards which you have. Stash/Hand Score: The sum of the rank of cards. 

- On the first turn you will be given 3 cards from the randomly shuffled deck. Eg: 3H, 2S, AD. Stash Score: 6
- In the next step you will have to either pick a card from pile or the deck. On the first turn, the top card of the deck is on the pile. The card on the pile is visible to you. So Card on the Pile : 2D . You have to decide whether you have to pick this card or you have to pick from pile. Since you alread
- Following the pickup step, you will have to drop one card from the stash/hand. Your Hand : 3H, 2S, AD, 2D
- You can drop any of these cards, but remember if you drop 2S or 2D you might not be able to make a series. So you have to decide whether to drop 3H or AD. Since there are maximum 20 turns, you have to keep your score low as well, Higher the score the higher points you lose. So you decide to drop 3H. Your hand/stash will be: 2S, 2D, AD
- Your opponent will play his hand, he/she shall pick up a card from a pile or deck and the drop a card to the pile.
- Now its your turn. The card on the Pile : 5H. Since picking up this card won't help you much. You decide to pick a card from the Deck. You get lucky, you get a 2C card. Your hand now is 2S, 2D, 2C, AD. Hurray! You can now meld these cards and drop the last card to make your score 0.
- You win this round. Your opponent Stash/Hand score : 15. So you are at 0 points and your opponent is at 15 points. You are ahead of him since you have to keep your score as low as possible.


## Rummy Environment

Importing Libraries

In [13]:
import random 
from functools import reduce
from collections import defaultdict
import numpy as np
from copy import copy
%matplotlib inline

Here, we define the SUITS, RANKS of the cards and their RANK_VALUE.

In [14]:
SUIT = ['H','S','D','C']
RANK = ['A', '2', '3', '4', '5','6','7']
RANK_VALUE = {'A': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'Q': 10, 'K': 10}

## Card Class Definition
__init__  : Defines the card details such as rank, suit and calculates the rank value


In [15]:
class Card :
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        self.rank_to_val = RANK_VALUE[self.rank]
    
    def __str__(self):
        return f'{self.rank}{self.suit}'

    def __repr__(self):
        return f'{self.rank}{self.suit}'

    
    def __eq__(self,other):
        return self.rank == other.rank and self.suit == other.suit
  
# Deck class contains some basic operations performed with the cards:
# 1. Shuffling the cards.
# 2. Drawing card from the deck.

## Deck Class Definition
__shuffle__ : Shuffles the deck in random order

__draw_card__ : Draws a card from the top of the deck

In [16]:
class Deck:
    def __init__(self,packs):
        self.packs = packs
        self.cards = []
        for pack in range(0,packs) :
            for suit in SUIT :
                for rank in RANK :
                    self.cards.append(Card(rank,suit))
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def draw_card(self):
        card = self.cards[0]
        self.cards.pop(0)
        return card

## Player Class:

### 1.__init__(self,name,stash=list(),isBot=False): 
Initializing stash, name, isBot/dealer points for each player.

### 2. deal_card(self,card):
This method appends the card in the stash and check the condition that length of stash should not be greater than nuber of cards length in game.

### 3. drop_card(self,card):
This method removes the card from stash and add that card into pile.

### 4. meld(self):
This method tries to find the cards with the same rank in the hand. If it finds then it will merge the cards in the hand to the melded cards array in the game. 

### 5. stash_score(self):
This method calculates sum of all the cards in stash according to the rank of each card.

### 6. get_info(self,debug):
This function fetch all the information of the player.

In [17]:
class Player:
  
    """  
        Player class to create a player object.
        eg: player = Player("player1", list(), isBot = False)
        Above declaration will be for your agent.
        All the player names should be unique or else you will get error.
        
    """

    def __init__(self,name,stash=list(),isBot=False,points=0,conn=None):
        self.stash = stash
        self.name = name
        self.game = None
        self.isBot = isBot
        self.points = points
        self.conn = conn
        
    def deal_card(self,card):
        try :
            self.stash.append(card)
            if len(stash) > self.game.cardsLength + 1 :
                raise ValueError('Cannot have cards greater than ')
        except ValueError as err:
            print(err.args)
            
    def drop_card(self,card):
        self.stash.remove(card)
        self.game.add_pile(card)
        return -1

    
    def meld(self):
        card_hash = defaultdict(list)
        for card in self.stash:
            card_hash[card.rank].append(card)
        melded_card_ranks = []
        for (card_rank,meld_cards) in card_hash.items():
            if len(meld_cards) >= 3 :
                self.game.meld.append(meld_cards)
                melded_card_ranks.append(card_rank)
                for card in meld_cards:
                    self.stash.remove(card)
        
        for card_rank in melded_card_ranks :
            card_hash.pop(card_rank)
        return len(melded_card_ranks) > 0

    def stash_score(self) :
        score = 0
        for card in self.stash :
            score += RANK_VALUE[card.rank]
        return score
    
    def get_info(self,debug):
        if debug :
            print(f'Player Name : {self.name} \n Stash Score: {self.stash_score()} \n Stash : {", ".join(str(x) for x in self.stash)}')
        card_ranks = []
        card_suits = []
        pileset = None
        pile = None
        for card in self.stash :
            card_suits.append(RANK_VALUE[card.rank])
            card_ranks.append(card.suit)
        if len(self.game.pile) > 0 : 
            return {"Stash Score" : self.stash_score(), "CardSuit":  card_suits, "CardRanks": card_ranks, "PileRank": self.game.pile[-1].rank, "PileSuit":self.game.pile[-1].suit}
        return {"Stash Score" : self.stash_score(), "CardSuit":  card_suits, "CardRanks": card_ranks}


## Game Enviroment:

### 1. __init():  

### 2. add_pile(self, card):  This method takes a card as argument and first checks number of cards in the deck. If its is ‘0’ then add the cards from file to deck and append the passed card to the pile.

### 3. pick_card(self, player, action):  This methods helps player picking up the card from either Pile or Deck based on action.
			We have defined, If action = 0 then, player will pick a card from Pile
			And if action. = 1 then player will pick card from Deck. 
			Meld condition will be check after player picks the card, if the meld condition satisfied, player won. 
            You can modify the rewards in return, only the values

### 4. pick_from_pile(self, player): This method helps player picking card from the pile and simultaneously a card from pile gets reduced.

### 5. pick_from_deck(self, player): This method is similar to above method but it helps picking up card from the Deck. 

### 6. get_player(self, player_name): This function fetch the details of the player given player_name.

### 7. computer_play(self, player): This method defines the play of the computer/Dealer in following sequence:
			--> Randomly taking actions from picking up card from deck/pile.
			--> Checking the meld condition afterwards.
			--> If the meld condition does not satisfied, remove the card from his stash.

### 8. play(self): This method defines all the function city of play for the player:
		  --> Decrementing the maximum number of turns defined per game.
		  --> For each player, it will check the 'stash', if the 'Stash' for any player = 0 (That player won), it will add the value of each card in stash for all other players.
		  --> Or If maximum number of turns in each round becomes 0, it will add the card values in stash for all the players and return.
      
### 9. drop_card(self,player,card) This method drops a card from the player given player object and card object and returns the reward associated with it.

      --> You can modify the rewards which gets returned in the value.


### 9. reset(self, players): This method will reinitialize all the deck, pile and players.

### 10. _update_turn(self): This method is used to count the number of turns in the game.

In [18]:
class RummyAgent() :
    """
    Simple Rummy Environment
    
    Simple Rummy is a game where you need to make all the cards in your hand same before your opponent does.
    Here you are given 3 cards in your hand/stash to play.
    For the first move you have to pick a card from the deck or from the pile. 
    The card in deck would be random but you can see the card from the pile.
    In the next move you will have to drop a card from your hand.
    Your goal is to collect all the cards of the same rank. 
    Higher the rank of the card, the higher points you lose in the game. 
    You need to keep the stash score low. Eg, if you can AH,7S,5D your strategy would be to either find the first pair of the card or by removing the highest card in the deck.
    You only have 20 turns to either win the same or collect low scoring card.
    You can't see other players cards or their stash scores.
    
    Parameters
    ====
    players: Player objects which will play the game.
    max_card_length : Number of cards each player can have
    max_turns: Number of turns in a rummy game
    """

    def __init__(self,players,max_card_length=5,max_turns=20) :
        self.max_card_length = max_card_length
        self.max_turns = max_turns
        self.reset(players)
        
    def update_player_cards(self,players):
        for player in players :
            player = Player(player.name,list(),isBot=player.isBot,points=player.points,conn=player.conn)
            stash = []
            for i in range(self.max_card_length):
                player.stash.append(self.deck.draw_card())
            player.game = self
            self.players.append(player)
        self.pile = [self.deck.draw_card()]

    def add_pile(self,card):
        if len(self.deck.cards) == 0 :
            self.deck.cards.extend(self.pile)
            self.deck.shuffle()
            self.pile = []
        self.pile.append(card)
        
        
    def pick_card(self,player,action):
        if action == 0:
            self.pick_from_pile(player)
        else :
            self.pick_from_deck(player)
        if player.meld() :
            return {"reward" : 10}
        else :
            return {"reward" : -1 * player.stash_score()}
       
    def pick_from_pile(self,player):
        card = self.pile[-1]
        self.pile.pop()
        return player.stash.append(card)
     
    def pick_from_deck(self,player):
        return player.stash.append(self.deck.draw_card())
    
    def get_player(self,player_name):
        return_player = [player for player in self.players if player.name == player_name]
        if len(return_player) != 1:
            print("Invalid Player")
            return None
        else:
            return return_player[0]
    
    def drop_card(self,player,card):
        player.drop_card(card)
        return {"reward" : -1 * player.stash_score()}
    
  
    def computer_play(self,player):
        #Gets a card from deck or pile
        if random.randint(0,1) == 1 :
            self.pick_from_pile(player)
        else :
            self.pick_from_deck(player)
            
        #tries to meld if it can
#         if random.randint(0,10) > 5 :
        player.meld()
        
        #removes a card from the stash
        if len(player.stash) != 0:
            card = player.stash[(random.randint(0,len(player.stash) - 1))]
            player.drop_card(card)
        
    def play(self):
        for player in self.players :
            if len(player.stash) == 0 :
                return True
        if self.max_turns <= 0 :
            return True
        return False

    def _update_turn(self):
        self.max_turns -= 1  
    
    def reset(self,players,max_turns=20):
        self.players = []
        self.deck = Deck(1)
        self.deck.shuffle()
        self.meld = []
        self.pile = []
        self.max_turns = max_turns
        self.update_player_cards(players)




### Example Run with Random Players

This following code shows how to create an environment and players to play 10 games with 20 maximum rounds. This only includes random picking up of card and dropping.  

In [19]:
p1 = Player('tabish',list())
p2 = Player('comp1',list(),isBot=True)
rummy = RummyAgent([p1,p2],max_card_length=3,max_turns=20)

In [20]:
maxiter = 3
debug = True
for j in range(maxiter):
        for player in rummy.players :
            player.points = player.stash_score()
        
        rummy.reset(rummy.players)
        random.shuffle(rummy.players)
        # int i = 0 
        if debug :
            print(f'**********************************\n\t\tGame Starts : {j}\n***********************************')
        while not rummy.play() :    
            rummy._update_turn()
            print(rummy.max_turns)       
            for player in rummy.players:
                if player.isBot :
                    if rummy.play():
                        continue
                    if debug :
                        print(f'{player.name} Plays')
                    rummy.computer_play(player)
                    if debug :
                        player.get_info(debug)
                        if player.stash == 0 :
                            print(f'{player.name} wins the round')

                else :
                    if rummy.play() :
                        continue
                    if debug :
                        print(f'{player.name} Plays')
                    player_info = player.get_info(debug)
                    action_taken = np.random.choice(1)
                    if debug :
                        print(f'Card in pile {player_info["PileSuit"]}{player_info["PileRank"]}')
                    result_1 = rummy.pick_card(player,action_taken)
                    result_1 = result_1["reward"]

                    if debug : 
                        print(f'{player.name} takes action {action_taken}')
                    #player stash will have no cards if the player has melded them 
                    #When you have picked up a card and you have drop it since the remaining cards have been melded. 
                    if len(player.stash) == 1:
                        rummy.drop_card(player,player.stash[0])
                        if debug :
                            print(f'{player.name} Wins the round')

                    elif len(player.stash) != 0 :

                        player_info = player.get_info(debug)
                        s = player_info['CardRanks']
                        action_taken = np.random.choice(4)
                        card = player.stash[action_taken]
                        if debug :
                            print(f'{player.name} drops card {card}')

                        result_1 = rummy.drop_card(player,card)
                        result_1 = result_1["reward"]
#                             pdb.set_trace()
                    else :
                        if debug :
                            print(f'{player.name} Wins the round')
                    if debug :
                        player.get_info(debug)
        


**********************************
		Game Starts : 0
***********************************
19
comp1 Plays
Player Name : comp1 
 Stash Score: 10 
 Stash : 5S, 4S, AH
tabish Plays
Player Name : tabish 
 Stash Score: 13 
 Stash : 5D, 2C, 6D
Card in pile C7
tabish takes action 0
Player Name : tabish 
 Stash Score: 20 
 Stash : 5D, 2C, 6D, 7C
tabish drops card 5D
Player Name : tabish 
 Stash Score: 15 
 Stash : 2C, 6D, 7C
18
comp1 Plays
Player Name : comp1 
 Stash Score: 11 
 Stash : 5S, AH, 5H
tabish Plays
Player Name : tabish 
 Stash Score: 15 
 Stash : 2C, 6D, 7C
Card in pile S4
tabish takes action 0
Player Name : tabish 
 Stash Score: 19 
 Stash : 2C, 6D, 7C, 4S
tabish drops card 6D
Player Name : tabish 
 Stash Score: 13 
 Stash : 2C, 7C, 4S
17
comp1 Plays
Player Name : comp1 
 Stash Score: 16 
 Stash : 5S, 5H, 6D
tabish Plays
Player Name : tabish 
 Stash Score: 13 
 Stash : 2C, 7C, 4S
Card in pile HA
tabish takes action 0
Player Name : tabish 
 Stash Score: 14 
 Stash : 2C, 7C, 4S, AH
ta

# III. Methods

- Decide your TD learning approach: SARSA or Q-learning? 
- Decide your function approximator.
- Describe your approach and the reason why you select it.
- Finish epsilon_greedy function and other TODOs. Explain it.
- Explain your codes.

# Review of the SARSA and Q-Learning

SARSA stands for State-Action-Reward-State-Action. In SARSA, the agent starts in state 1, performs action 1, and gets a reward (reward 1). Now, it’s in state 2 and performs another action (action 2) and gets the reward from this state (reward 2) before it goes back and updates the value of action 1 performed in state 1. 

We call this as *on-policy control*, or **SARSA**. 

$$
    Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha ( R_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)) 
$$


In contrast, in Q-learning the agent starts in state 1, performs action 1 and gets a reward (reward 1), and then looks and sees what the maximum possible reward for an action is in state 2, and uses that to update the action value of performing action 1 in state 1. So the difference is in the way the future reward is found. In Q-learning it’s simply the highest possible action that can be taken from state 2, and in SARSA it’s the value of the actual action that was taken.

$$
    Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha ( R_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t)) 
$$


## Choice of TD learning and Reason: Q-LEARNING

SARSA and Q Learning are both reinforcement learning algorithms that work in a similar way. The most striking difference is that SARSA is on policy while Q Learning is off policy. 

Q-learning is better than SARSA, as it is everytime use the greedy one i.e. is the optimal step at every step of the game and proceeds further. This leads to the better results when compared to the SARSA.

SARSA will approach convergence allowing for possible penalties from exploratory moves, whilst Q-learning will ignore them. That makes SARSA more conservative - if there is risk of a large negative reward close to the optimal path, Q-learning will tend to trigger that reward whilst exploring, whilst SARSA will tend to avoid a dangerous optimal path and only slowly learn to use it when the exploration parameters are reduced. 

## Choice of Function Approximation and Reason:

I have use Epsilon greedy and greedy approaches here. Epsilon greedy will check for the random avlue genetrated is less than the given epsilon or not and proceed futher.

If the epsilon is greater than random generated values then it will take random action.

Or else it will take th greedy i.e the optimal step and proceed with the game.

I have chnaged the parameters as well and tested the outputs. When I changed the gamma , alpha and epsilon values, I coudl see that the number of times the player is winning is kept on changing.

## RLAgent for Rummy

Referencing the RLAgent in the lecture note, make your own agent class for the game. 

In [21]:
# p1 = Player('tabish',list())
# p2 = Player('comp1',list(),isBot=True)
# rummy = RLAgent([p1,p2],max_card_length=3,max_turns=20)

In [22]:
class RLAgent:
    """
        Reinforcement Learning Agent Model for training/testing
        with Tabular function approximation

    """

    def __init__(self):
        # Initialize the self.Q Table with states and actions
        self.Q1 = np.zeros((8, 8, 8, 8, 2))
        self.Q1[...] = 0
        for i in range(7):
            i=i+1
            self.Q1[:, i, i, i, 0] = 2
            self.Q1[i, :, i, i, 0] = 2
            self.Q1[i, i, :, i, 0] = 2
    
            
        self.Q2 = np.zeros((8, 8, 8, 8, 4))
        self.Q2[...] = 0
        for i in range(7):
            i=i+1
            self.Q2[:, i, i, i, 0] = 2
            self.Q2[i, :, i, i, 1] = 2
            self.Q2[i, i, :, i, 2] = 2
            self.Q2[i, i, i, :, 3] = 2

    def epsilon_greedy1(self, epsilon, s):
        if np.random.uniform() < epsilon:
            action = np.random.choice(2)
            return action
        else :
            i_max = np.where(self.Q1[s[0], s[1],s[2],s[3], :] == np.max(self.Q1[s[0], s[1],s[2],s[3], :]))[0]
            action = int(np.random.choice(i_max))
            return action
        
    def epsilon_greedy2(self, epsilon, s):
        if np.random.uniform() < epsilon:
            action = np.random.choice(4)
            return action
        else :
            i_max = np.where(self.Q2[s[0], s[1],s[2],s[3], :] == np.max(self.Q2[s[0], s[1],s[2],s[3], :]))[0]
            action = int(np.random.choice(i_max))
            return action
            

    def train(self,gamma, alpha, epsilon, number_of_games, number_of_rounds):
        # parameters
        self.epsilon=epsilon
        self.gamma = gamma
        self.alpha = alpha
        self.number_of_games = number_of_games
        self.number_of_rounds = number_of_rounds
        rtrace=[]
        
        player1 = Player('comp1', list(),isBot = True)
        player2 = Player('player1', list())
        
        self.rummy = RummyAgent([player1, player2],3,self.number_of_rounds)
       
        
        debug = False
        botwins=0
        anusha=0

        for j in range(self.number_of_games):
            for player in self.rummy.players:
                player.points = player.stash_score()

            self.rummy.reset(self.rummy.players)
            random.shuffle(self.rummy.players)
            rewards = []

            if debug:
                print(f'**********************************\n\t\tGame Starts : {j}\n***********************************')
            while not self.rummy.play():
    
                self.rummy._update_turn()

                for player in self.rummy.players:
                    if player.isBot:
                        if self.rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        self.rummy.computer_play(player)
                        if player.stash == 0:
                                botwins=botwins + 1
                        if debug:
                            player.get_info(debug)
                            if player.stash == 0:
                                botwins=botwins + 1
                                print(f'{player.name} wins the round')
                    else:
                        if self.rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        player_info = player.get_info(debug)
                        if debug:
                            print(f'Card in pile {player_info["PileSuit"]}{player_info["PileRank"]}')
                        state = player_info["CardSuit"]
                        state.append(RANK_VALUE[player_info["PileRank"]])
                        action_taken = self.epsilon_greedy1(self.epsilon, state)
#                        self.Q1[state[0]][state[1]][state[2]] [state[3]][action_taken] = self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken] + self.alpha * (result + (self.gamma * np.max(self.Q1[state[1]][state[2]][state[3]][:])) -self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken])
                        if action_taken == 0 :  
                            result = self.rummy.pick_card(player, 0)
                            result = result["reward"]
                            rewards.append(result)
                        else:
                            result = self.rummy.pick_card(player, 1)
                            result = result["reward"]
                            rewards.append(result)
                        self.Q1[state[0]][state[1]][state[2]] [state[3]][action_taken] = self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken] + self.alpha * (result + (self.gamma * np.max(self.Q1[state[1]][state[2]][state[3]][:])) -self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken])      
 

                        player_info = player.get_info(debug)
                        if len(player.stash) == 1:
                            self.rummy.drop_card(player, player.stash[0])
                            anusha = anusha +1
                            if debug:
                                anusha = anusha +1
                                print(f'{player.name} Wins the round')

                        elif len(player.stash) != 0 :
                            state2 = player_info["CardSuit"]
                            action_taken = self.epsilon_greedy2(self.epsilon, state2)
                            card = player.stash[action_taken]
                            result = self.rummy.drop_card(player, card)
                            result = result["reward"]
                            rewards.append(result)
                            self.Q2[state2[0]][state2[1]][state2[2]][state2[3]][action_taken] = \
                                self.Q2[state2[0]][state2[1]][state2[2]][state2[3]][action_taken] + self.alpha * (
                                            result + (self.gamma * np.max(self.Q2[state2[1]][state2[2]][state2[3]][:])) -
                                            self.Q2[state2[0]][state2[1]][state2[2]][state2[3]][action_taken])
                        else:
                            anusha = anusha +1
                            if debug:
                                anusha = anusha +1
                                print(f'{player.name} Wins the round')

                        if debug:
                            print(f'{player.name} takes action {action_taken}')
                        if debug:
                            player.get_info(debug)
            rtrace.append(np.sum(rewards))                  
        return anusha,botwins,rtrace,self.Q1,self.Q2
                                  
    def test(self):
        
        self.number_of_games = 4
        self.number_of_rounds = 20
        
        player1 = Player('comp1', list(),isBot = True)
        player2 = Player('player1', list())
        
        self.rummy = RummyAgent([player1, player2],3,self.number_of_rounds)
       
        
        debug = True
        botwins=0
        anusha=0

        for j in range(self.number_of_games):
            for player in self.rummy.players:
                player.points = player.stash_score()

            self.rummy.reset(self.rummy.players)
            random.shuffle(self.rummy.players)

            if debug:
                print(f'**********************************\n\t\tGame Starts : {j}\n***********************************')
            while not self.rummy.play():
    
                self.rummy._update_turn()

                for player in self.rummy.players:
                    if player.isBot:
                        if self.rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        self.rummy.computer_play(player)
                        if player.stash == 0:
                                botwins=botwins + 1
                        if debug:
                            player.get_info(debug)
                            if player.stash == 0:
                                botwins=botwins + 1
                                print(f'{player.name} wins the round')
                    else:
                        if self.rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        player_info = player.get_info(debug)
                        if debug:
                            print(f'Card in pile {player_info["PileSuit"]}{player_info["PileRank"]}')
                        state = player_info["CardSuit"]
                        state.append(RANK_VALUE[player_info["PileRank"]])
                        action_taken = self.epsilon_greedy1(self.epsilon, state)
#                        self.Q1[state[0]][state[1]][state[2]] [state[3]][action_taken] = self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken] + self.alpha * (result + (self.gamma * np.max(self.Q1[state[1]][state[2]][state[3]][:])) -self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken])
                        if action_taken == 0 :  
                            result = self.rummy.pick_card(player, 0)
#                             result = result["reward"]
                        else:
                            result = self.rummy.pick_card(player, 1)
#                             result = result["reward"]                         
#                         self.Q1[state[0]][state[1]][state[2]] [state[3]][action_taken] = self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken] + self.alpha * (result + (self.gamma * np.max(self.Q1[state[1]][state[2]][state[3]][:])) -self.Q1[state[0]][state[1]][state[2]][state[3]][action_taken])      
 

                        player_info = player.get_info(debug)
                        if len(player.stash) == 1:
                            self.rummy.drop_card(player, player.stash[0])
                            anusha = anusha +1
                            if debug:
                                anusha = anusha +1
                                print(f'{player.name} Wins the round')

                        elif len(player.stash) != 0 :
                            state2 = player_info["CardSuit"]
                            action_taken = self.epsilon_greedy2(self.epsilon, state2)
                            card = player.stash[action_taken]
                            result = self.rummy.drop_card(player, card)
#                             result = result["reward"]
#                             self.Q2[state2[0]][state2[1]][state2[2]][state2[3]][action_taken] = \
#                                 self.Q2[state2[0]][state2[1]][state2[2]][state2[3]][action_taken] + self.alpha * (
#                                             result + (self.gamma * np.max(self.Q2[state2[1]][state2[2]][state2[3]][:])) -
#                                             self.Q2[state2[0]][state2[1]][state2[2]][state2[3]][action_taken])
                        else:
                            anusha = anusha +1
                            if debug:
                                anusha = anusha +1
                                print(f'{player.name} Wins the round')

                        if debug:
                            print(f'{player.name} takes action {action_taken}')
                        if debug:
                            player.get_info(debug)
                            
        
                                  

### Observation 1 : gamma=0.95, alpha=0.1, epsilon=0.01

In [23]:
environment=RLAgent()

In [24]:
t1,t2,trace,Q1,Q2= environment.train(gamma=0.95, alpha=0.1, epsilon=0.01, number_of_games=20000, number_of_rounds=400)

In [25]:
print(t1,t2)

13106 0


In [26]:
environment.test()

**********************************
		Game Starts : 0
***********************************
comp1 Plays
Player Name : comp1 
 Stash Score: 10 
 Stash : 4H, 5S, AS
player1 Plays
Player Name : player1 
 Stash Score: 7 
 Stash : 2C, AH, 4S
Card in pile C7
Player Name : player1 
 Stash Score: 8 
 Stash : 2C, AH, 4S, AC
player1 takes action 2
Player Name : player1 
 Stash Score: 4 
 Stash : 2C, AH, AC
comp1 Plays
Player Name : comp1 
 Stash Score: 13 
 Stash : 5S, AS, 7D
player1 Plays
Player Name : player1 
 Stash Score: 4 
 Stash : 2C, AH, AC
Card in pile H4
Player Name : player1 
 Stash Score: 8 
 Stash : 2C, AH, AC, 4H
player1 takes action 3
Player Name : player1 
 Stash Score: 4 
 Stash : 2C, AH, AC
comp1 Plays
Player Name : comp1 
 Stash Score: 13 
 Stash : 5S, AS, 7D
player1 Plays
Player Name : player1 
 Stash Score: 4 
 Stash : 2C, AH, AC
Card in pile H4
Player Name : player1 
 Stash Score: 8 
 Stash : 2C, AH, AC, 4H
player1 takes action 3
Player Name : player1 
 Stash Score: 4 
 Stash 

In [27]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(np.arange(20000),trace)
plt.xlabel("Games")
plt.ylabel("Rewards per game")

### Observation 2 : gamma=0.75, alpha=0.1, epsilon=0.1

In [ ]:
environment=RLAgent()

In [ ]:
t1,t2,trace,q3,q4= environment.train(gamma=0.75, alpha=0.1, epsilon=0.1, number_of_games=20000, number_of_rounds=400)

In [ ]:
print(t1,t2)

In [ ]:
environment.test()

In [ ]:
plt.plot(np.arange(20000),trace)
plt.xlabel("Games")
plt.ylabel("Rewards per game")

### Observation 3 : gamma=0.75, alpha=0.1, epsilon=0.001

In [ ]:
environment=RLAgent()

In [ ]:
t1,t2,trace,q5,q6= environment.train(gamma=0.75, alpha=0.1, epsilon=0.001, number_of_games=20000, number_of_rounds=400)

In [ ]:
print(t1,t2)

In [ ]:
environment.test()

In [ ]:
plt.plot(np.arange(20000),trace,"y")
plt.xlabel(" Number of Games")
plt.ylabel("Rewards")
plt.title("Rewards Plot")

# IV - Results

-  Different plots are drawn based on different values for parameters gamma,alpha and epsilon.As epsilon values decreases we can the see the number of games won increases that is we get a better output.
- We have plotted the rewards for each game.As the number of games increase the rewards tend to be positive side meaning the system is being trained.

# V. Conclusions

- The most challenging part was the implementatio of q table.I have tried implementing through a single Q table but failed in the process.Hence I have chosen to take two Q table approach.

# Tournament 

In [ ]:
import pickle

In [ ]:
# with open('anusha.pkl', 'rb') as f:
#     Q = pickle.load(f)

In [28]:
def epsilon_greedy1(epsilon,s):
    if np.random.uniform() < epsilon:
        action = np.random.choice(2)
        return action
    else :
        i_max = np.where(Q1[s[0], s[1],s[2],s[3], :] == np.max(Q1[s[0], s[1],s[2],s[3], :]))[0]
        action = int(np.random.choice(i_max))
        return action

def epsilon_greedy2(epsilon, s):
    if np.random.uniform() < epsilon:
        action = np.random.choice(4)
        return action
    else :
        i_max = np.where(Q2[s[0], s[1],s[2],s[3], :] == np.max(Q2[s[0], s[1],s[2],s[3], :]))[0]
        action = int(np.random.choice(i_max))
        return action

In [29]:
Q1,Q2

(array([[[[[  0.        ,   0.        ],
           [  0.        ,   0.        ],
           [  0.        ,   0.        ],
           ...,
           [  0.        ,   0.        ],
           [  0.        ,   0.        ],
           [  0.        ,   0.        ]],
 
          [[  0.        ,   0.        ],
           [  0.        ,   0.        ],
           [  0.        ,   0.        ],
           ...,
           [  0.        ,   0.        ],
           [  0.        ,   0.        ],
           [  0.        ,   0.        ]],
 
          [[  0.        ,   0.        ],
           [  0.        ,   0.        ],
           [  0.        ,   0.        ],
           ...,
           [  0.        ,   0.        ],
           [  0.        ,   0.        ],
           [  0.        ,   0.        ]],
 
          ...,
 
          [[  0.        ,   0.        ],
           [  0.        ,   0.        ],
           [  0.        ,   0.        ],
           ...,
           [  0.        ,   0.        ],
        

In [30]:
import pickle
with open('anusha.pkl', 'wb') as f:
    pickle.dump(Q1,f)

In [34]:
import pickle
with open('anusha1.pkl', 'rb') as f:
    Q1=pickle.load(f)

In [35]:
with open('anusha1.pkl', 'rb') as f:
    Q2=pickle.load(f)

In [36]:
Q1,Q2

(array([[[[[  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ],
           ...,
           [  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ]],
 
          [[  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ],
           ...,
           [  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.        ]],
 
          [[  0.        ,   0.        ,   0.        ,   0.        ],
           [  0.        ,   0.        ,   0.        ,   0.       

In [37]:
import socket
import os
import subprocess
import ast
import pdb
import numpy as np
import time
player = 'avudathu' #enter unique name for your game
s = socket.socket() 
# s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
host = '100.24.254.170' # change ip addres to ip address of your computer or use 'localhost' to practice
port = 9999
ep=0.01
s.connect((host, port))
while True:
    data = s.recv(1024)
    if data.decode("utf-8") == 'send':
        s.send(str.encode( player))
        client_response = str(s.recv(20480), "utf-8")
        print(client_response, end="")
        break

while True:
    data = s.recv(20480)
    print(data,end=".")
    if data.decode("utf-8") == f'sendinfo':
        s.send(str.encode("acknowledged_pick"))
        ##Here you will recieve information about the player such as card rank, card suit, stash score, 
        client_response = str(s.recv(20480), "utf-8")
        print(client_response)
        ro=ast.literal_eval(client_response)
        state=ro['CardSuit']
        state.append(int(player_info["PileRank"]))
        action=epsilon_greedy1(ep,state)
#         action = np.random.randint(0, 2)
        print("action pick ",action)
        s.send(str.encode( str(action)))
    if data.decode("utf-8") == f'sendaction_for_card_drop':
        s.send(str.encode("acknowledged_drop"))
        ro=str(s.recv(20480), "utf-8")
        ro=ast.literal_eval(ro)
        print("action send for drop")
        print(ro)
        #follow same procedure as abov to send observations to your agent
        ###### Look at the above observations and action in variable action#####
        state = ro["CardSuit"]
        if len(state)==1:
            action=0
        else:
            action = epsilon_greedy2(ep,state)
#         action = np.random.randint(0, 3)
        s.send(str.encode( str(action)))
        print("action drop",action)
    if data.decode("utf-8") == 'gameover':
        print("Game over wait for others to play and wait for results")
        break
    if data.decode("utf-8") == 'round_over':
        s.send(str.encode("acknowledged_round_over"))
        print('Round over')
        client_response = str(s.recv(20480), "utf-8")
        print(client_response, end="")


Connection established, Names collectedb'sendinfo'.{'Stash Score': 10, 'CardSuit': [6, 2, 2], 'CardRanks': ['H', 'S', 'C'], 'PileRank': '4', 'PileSuit': 'S'}
action pick  1
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 15, 'CardSuit': [6, 2, 2, 5], 'CardRanks': ['H', 'S', 'C', 'C'], 'PileRank': '4', 'PileSuit': 'S'}
action drop 2
b'sendinfo'.{'Stash Score': 13, 'CardSuit': [6, 2, 5], 'CardRanks': ['H', 'S', 'C'], 'PileRank': '3', 'PileSuit': 'S'}
action pick  0
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 16, 'CardSuit': [6, 2, 5, 3], 'CardRanks': ['H', 'S', 'C', 'S'], 'PileRank': 'A', 'PileSuit': 'S'}
action drop 0
b'sendinfo'.{'Stash Score': 10, 'CardSuit': [2, 5, 3], 'CardRanks': ['S', 'C', 'S'], 'PileRank': '3', 'PileSuit': 'H'}
action pick  2
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 16, 'CardSuit': [2, 5, 3, 6], 'CardRanks': ['S', 'C', 'S', 'D'], 'PileRank': '3', 'PileSuit': 'H'}
action drop 3
b'sendinfo'.{'Stash Score

b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 11, 'CardSuit': [3, 2, 2, 4], 'CardRanks': ['H', 'D', 'H', 'C'], 'PileRank': '6', 'PileSuit': 'C'}
action drop 3
b'sendinfo'.{'Stash Score': 7, 'CardSuit': [3, 2, 2], 'CardRanks': ['H', 'D', 'H'], 'PileRank': '3', 'PileSuit': 'D'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 8, 'CardSuit': [3, 2, 2, 1], 'CardRanks': ['H', 'D', 'H', 'C'], 'PileRank': '3', 'PileSuit': 'D'}
action drop 0
b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 2, 1], 'CardRanks': ['D', 'H', 'C'], 'PileRank': '6', 'PileSuit': 'S'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 9, 'CardSuit': [2, 2, 1, 4], 'CardRanks': ['D', 'H', 'C', 'S'], 'PileRank': '6', 'PileSuit': 'S'}
action drop 3
b'round_over'.Round over
Player: comp3  Score: 32 Stash Score : 0
Player: comp2  Score: 40 Stash Score : 8
Player: avudathu  Score: 17 Stash Score : 5
Player: gsilewar  Score: 34 Stash Score : 12
b'send

b'sendinfo'.{'Stash Score': 13, 'CardSuit': [5, 5, 3], 'CardRanks': ['S', 'H', 'D'], 'PileRank': '2', 'PileSuit': 'D'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 20, 'CardSuit': [5, 5, 3, 7], 'CardRanks': ['S', 'H', 'D', 'C'], 'PileRank': '2', 'PileSuit': 'D'}
action drop 2
b'sendinfo'.{'Stash Score': 17, 'CardSuit': [5, 5, 7], 'CardRanks': ['S', 'H', 'C'], 'PileRank': 'A', 'PileSuit': 'H'}
action pick  2
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 23, 'CardSuit': [5, 5, 7, 6], 'CardRanks': ['S', 'H', 'C', 'S'], 'PileRank': 'A', 'PileSuit': 'H'}
action drop 2
b'sendinfo'.{'Stash Score': 16, 'CardSuit': [5, 5, 6], 'CardRanks': ['S', 'H', 'S'], 'PileRank': '7', 'PileSuit': 'D'}
action pick  1
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 22, 'CardSuit': [5, 5, 6, 6], 'CardRanks': ['S', 'H', 'S', 'C'], 'PileRank': '7', 'PileSuit': 'D'}
action drop 2
b'sendinfo'.{'Stash Score': 16, 'CardSuit': [5, 5, 6], 'CardRank

b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 1, 2], 'CardRanks': ['H', 'C', 'D'], 'PileRank': '6', 'PileSuit': 'C'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 8, 'CardSuit': [2, 1, 2, 3], 'CardRanks': ['H', 'C', 'D', 'C'], 'PileRank': '6', 'PileSuit': 'C'}
action drop 3
b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 1, 2], 'CardRanks': ['H', 'C', 'D'], 'PileRank': '7', 'PileSuit': 'H'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 10, 'CardSuit': [2, 1, 2, 5], 'CardRanks': ['H', 'C', 'D', 'H'], 'PileRank': '7', 'PileSuit': 'H'}
action drop 3
b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 1, 2], 'CardRanks': ['H', 'C', 'D'], 'PileRank': 'A', 'PileSuit': 'H'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 10, 'CardSuit': [2, 1, 2, 5], 'CardRanks': ['H', 'C', 'D', 'D'], 'PileRank': 'A', 'PileSuit': 'H'}
action drop 3
b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 1, 2], 'CardRanks': [

b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 2, 1], 'CardRanks': ['D', 'C', 'H'], 'PileRank': '7', 'PileSuit': 'H'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 12, 'CardSuit': [2, 2, 1, 7], 'CardRanks': ['D', 'C', 'H', 'D'], 'PileRank': '7', 'PileSuit': 'H'}
action drop 3
b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 2, 1], 'CardRanks': ['D', 'C', 'H'], 'PileRank': '6', 'PileSuit': 'D'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 10, 'CardSuit': [2, 2, 1, 5], 'CardRanks': ['D', 'C', 'H', 'S'], 'PileRank': '6', 'PileSuit': 'D'}
action drop 3
b'sendinfo'.{'Stash Score': 5, 'CardSuit': [2, 2, 1], 'CardRanks': ['D', 'C', 'H'], 'PileRank': '3', 'PileSuit': 'C'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 1, 'CardSuit': [1], 'CardRanks': ['H'], 'PileRank': '3', 'PileSuit': 'C'}
action drop 0
b'round_over'.Round over
Player: avudathu  Score: 63 Stash Score : 0
Player: comp3  Score: 1

b'sendinfo'.{'Stash Score': 11, 'CardSuit': [4, 6, 1], 'CardRanks': ['S', 'S', 'S'], 'PileRank': '3', 'PileSuit': 'S'}
action pick  1
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 15, 'CardSuit': [4, 6, 1, 4], 'CardRanks': ['S', 'S', 'S', 'H'], 'PileRank': '3', 'PileSuit': 'S'}
action drop 1
b'sendinfo'.{'Stash Score': 9, 'CardSuit': [4, 1, 4], 'CardRanks': ['S', 'S', 'H'], 'PileRank': 'A', 'PileSuit': 'H'}
action pick  2
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 10, 'CardSuit': [4, 1, 4, 1], 'CardRanks': ['S', 'S', 'H', 'D'], 'PileRank': 'A', 'PileSuit': 'H'}
action drop 2
b'round_over'.Round over
Player: gsilewar  Score: 128 Stash Score : 0
Player: avudathu  Score: 73 Stash Score : 6
Player: comp2  Score: 144 Stash Score : 17
Player: comp3  Score: 161 Stash Score : 8
b'sendinfo'.{'Stash Score': 8, 'CardSuit': [3, 4, 1], 'CardRanks': ['D', 'H', 'H'], 'PileRank': '3', 'PileSuit': 'H'}
action pick  1
b'sendaction_for_card_drop'.action send for d

b'sendinfo'.{'Stash Score': 6, 'CardSuit': [1, 3, 2], 'CardRanks': ['S', 'C', 'S'], 'PileRank': '3', 'PileSuit': 'S'}
action pick  1
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 13, 'CardSuit': [1, 3, 2, 7], 'CardRanks': ['S', 'C', 'S', 'S'], 'PileRank': '3', 'PileSuit': 'S'}
action drop 3
b'sendinfo'.{'Stash Score': 6, 'CardSuit': [1, 3, 2], 'CardRanks': ['S', 'C', 'S'], 'PileRank': '6', 'PileSuit': 'S'}
action pick  1
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 11, 'CardSuit': [1, 3, 2, 5], 'CardRanks': ['S', 'C', 'S', 'H'], 'PileRank': '6', 'PileSuit': 'S'}
action drop 3
b'sendinfo'.{'Stash Score': 6, 'CardSuit': [1, 3, 2], 'CardRanks': ['S', 'C', 'S'], 'PileRank': '7', 'PileSuit': 'C'}
action pick  3
b'sendaction_for_card_drop'.action send for drop
{'Stash Score': 13, 'CardSuit': [1, 3, 2, 7], 'CardRanks': ['S', 'C', 'S', 'H'], 'PileRank': '7', 'PileSuit': 'C'}
action drop 3
b'sendinfo'.{'Stash Score': 6, 'CardSuit': [1, 3, 2], 'CardRanks': 

# Extra Credit for Tournament Top16!

- In two classes, we will have qualification and tournament.
- To participate the tournament, you need to use this [client](http://nbviewer.jupyter.org/url/webpages.uncc.edu/mlee173/teach/itcs6156/notebooks/assign/StudentClient.ipynb) codes.
- Locally you can run the [server](http://nbviewer.jupyter.org/url/webpages.uncc.edu/mlee173/teach/itcs6156/notebooks/assign/SERVER.ipynb) to verify your run. 

### Qualification Round 

- In qualification game, individual will play against a fixed policy agent on the server
  - You can access the server: TBA
  
  - Highly ranked (by the profit) will proceed to the tournament.
  - max. 32 players will be cut for main tournament (by 11 pm on Dec 1st).
  
### Tournament

- In tournament, 4 players will be play in each round.
  - In a table game, two best scoring players will proceed to next round
  - In final, the game will ends in 50 rounds and winner will be the one with lowest balance. 

## Grading

We will test your notebook with an additional grid.txt file. Please make sure to test in different maze input files. 

points | | description
--|--|:--
5 | Overview| states the objective and the appraoch 
35 | Methods | 
 |10| Review of the SARSA and Q-Learning
 | 5| Choice of TD learning and Reason
 | 5| Choice of Function Approximation and Reason
 |10| Implementation of the selected approach (RLAgent)
 | 5| explanation of the codes
45 | Results 
 |10| Reports the selected parameters 
 |15| Experimental outputs that show the choice of parameters. How do you choose them?
 |10| Output plots (5 for each)
 |10| reading and analysis of learning results and plots
10 | Participation to the tournament.
5 | Conclusions 
